# Module 11 Challenge

## Deliverable 2: Scrape and Analyze Mars Weather Data

In [ ]:
# Import relevant libraries
from splinter import Browser
from bs4 import BeautifulSoup as soup
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
browser = Browser('chrome')

### Step 1: Visit the Website

Use automated browsing to visit the [Mars Temperature Data Site](https://static.bc-edx.com/data/web/mars_facts/temperature.html). Inspect the page to identify which elements to scrape.

   > **Hint** To identify which elements to scrape, you might want to inspect the page by using Chrome DevTools to discover whether the table contains usable classes.


In [ ]:
# Visit the website
url = "https://static.bc-edx.com/data/web/mars_facts/temperature.html"
browser.visit(url)

### Step 2: Scrape the Table

Create a Beautiful Soup object and use it to scrape the data in the HTML table.

Note that this can also be achieved by using the Pandas `read_html` function. However, use Beautiful Soup here to continue sharpening your web scraping skills.

In [ ]:
# Create a Beautiful Soup Object
html = browser.html
soup = soup(html, 'html.parser')

In [ ]:
# Print the HTML for reference
print(html)

In [ ]:
# Find the table
table = soup.find('table', class_='table')

In [ ]:
# Extract all rows of data
mars_temp_data = {}
rows = table.find_all('tr')

# Loop through the rows
if rows:
    # Extract header row separately
    header_row = rows[0]
    headers = [header.text.strip() for header in header_row.find_all('th')]

    # Loop through the data rows
    for row in rows[1:]:
        # Extract individual cells in each data row
        cells = row.find_all('td')

        # Check if there are cells in the row
        if cells:
            # Extract data from each cell and add to the dictionary
            row_data = [cell.text.strip() for cell in cells]
            row_dict = dict(zip(headers, row_data))
            
            # Use the 'id' value as the key
            mars_temp_data[row_dict['id']] = row_dict

# Display the extracted data
print(mars_temp_data)

### Step 3: Store the Data

Assemble the scraped data into a Pandas DataFrame. The columns should have the same headings as the table on the website. Here’s an explanation of the column headings:

* `id`: the identification number of a single transmission from the Curiosity rover
* `terrestrial_date`: the date on Earth
* `sol`: the number of elapsed sols (Martian days) since Curiosity landed on Mars
* `ls`: the solar longitude
* `month`: the Martian month
* `min_temp`: the minimum temperature, in Celsius, of a single Martian day (sol)
* `pressure`: The atmospheric pressure at Curiosity's location

In [ ]:
# Create an empty list
data_rows = []

# Loop through the scraped data to create a list of rows
for key, row_dict in mars_temp_data.items():
    data_rows.append(list(row_dict.values()))

In [ ]:
# Create a list of column names (headers)
column_names = list(mars_temp_data.values())[0].keys()

# Create a Pandas DataFrame by using the list of rows and the list of column names
mars_df = pd.DataFrame(data_rows, columns=column_names)

# Confirm DataFrame was created successfully
print(mars_df)

### Step 4: Prepare Data for Analysis

Examine the data types that are currently associated with each column. If necessary, cast (or convert) the data to the appropriate `datetime`, `int`, or `float` data types.

  > **Hint** You can use the Pandas `astype` and `to_datetime` methods to accomplish this task.


In [ ]:
# Examine data type of each column
mars_df.dtypes

In [ ]:
# Convert 'sol', 'ls', 'month', 'min_temp', and 'pressure' columns to the specified data types
mars_df['sol'] = mars_df['sol'].astype('int64')
mars_df['ls'] = mars_df['ls'].astype('int64')
mars_df['month'] = mars_df['month'].astype('int64')
mars_df['min_temp'] = mars_df['min_temp'].astype('float64')
mars_df['pressure'] = mars_df['pressure'].astype('float64')

# Convert 'terrestrial_date' to datetime type
mars_df['terrestrial_date'] = pd.to_datetime(mars_df['terrestrial_date'], errors='coerce')

# Display the updated data types
print(mars_df.dtypes)

### Step 5: Analyze the Data

Analyze your dataset by using Pandas functions to answer the following questions:

1. How many months exist on Mars?
2. How many Martian (and not Earth) days worth of data exist in the scraped dataset?
3. What are the coldest and the warmest months on Mars (at the location of Curiosity)? To answer this question:
    * Find the average the minimum daily temperature for all of the months.
    * Plot the results as a bar chart.
4. Which months have the lowest and the highest atmospheric pressure on Mars? To answer this question:
    * Find the average the daily atmospheric pressure of all the months.
    * Plot the results as a bar chart.
5. About how many terrestrial (Earth) days exist in a Martian year? To answer this question:
    * Consider how many days elapse on Earth in the time that Mars circles the Sun once.
    * Visually estimate the result by plotting the daily minimum temperature.


In [ ]:
# 1. How many months are there on Mars?
number_of_months = mars_df['month'].nunique()

# Display the result
print(f"There are {number_of_months} unique months on Mars.")

In [ ]:
# 2. How many Martian days' worth of data are there?
number_of_martian_days = mars_df['sol'].nunique()

# Display the result
print(f"There are {number_of_martian_days} Martian days in this data set.")

In [ ]:
# 3. What is the average low temperature by month?
average_low_temperature_by_month = mars_df.groupby('month')['min_temp'].mean()

# Display the result
average_low_temperature_by_month

In [ ]:
# Plot the average temperature by month
plt.figure(figsize=(10, 6))
plt.plot(average_low_temperature_by_month.index, average_low_temperature_by_month, marker='o', linestyle='-', color = 'green')
plt.title('Average Low Temperature on Mars by Month')
plt.xlabel('Month')
plt.ylabel('Average Low Temperature (°C)')
plt.show()

In [ ]:
# Identify the coldest and hottest months in Curiosity's location
coldest_month = average_low_temperature_by_month.idxmin()
hottest_month = average_low_temperature_by_month.idxmax()

# Get the corresponding temperatures
coldest_temperature = average_low_temperature_by_month.min()
hottest_temperature = average_low_temperature_by_month.max()

# Display the results
print(f"The coldest month is {coldest_month} with an average low temperature of {coldest_temperature} °C.")
print(f"The hottest month is {hottest_month} with an average low temperature of {hottest_temperature} °C.")

In [ ]:
# 4. Average pressure by Martian month
average_pressure_by_month = mars_df.groupby('month')['pressure'].mean()

# Display the result
average_pressure_by_month

In [ ]:
# Plot the average pressure by month
plt.figure(figsize=(10, 6))
plt.plot(average_pressure_by_month.index, average_pressure_by_month, marker='o', linestyle='-', color='green')
plt.title('Average Pressure on Mars by Month')
plt.xlabel('Month')
plt.ylabel('Average Pressure')
plt.show()

In [ ]:
# 5. How many terrestrial (earth) days are there in a Martian year?

# Plot minimum temperature against Martian days (sol)
plt.figure(figsize=(10, 6))
plt.plot(mars_df['sol'], mars_df['min_temp'], marker='o', linestyle='-', color='green')
plt.title('Minimum Temperature on Mars Over Martian Days')
plt.xlabel('Martian Days (sol)')
plt.ylabel('Minimum Temperature (°C)')
plt.grid(True)
plt.show()

# Measuring 'valley-to-valley' in the below chart will provide the approximate number of Martian days in a Earth year
# 1250 - 575 = 675

### Step 6: Save the Data

Export the DataFrame to a CSV file.

In [ ]:
# Write the data to a CSV
mars_df.to_csv("../Mars/Mars_df.csv", index_label=False)

In [ ]:
browser.quit()